# Extracting electrical features using BluePyEModel

This notebook will show how to use BluePyEModel to extract electrical features from experimental voltage traces. For this example we will use the rat somatosensory cortex data present here in the directory "exp_data" (Also available at: http://microcircuits.epfl.ch/#/article/article_4_eph)

For a more detailed used-case of BluePyEfe 2, read the example present in the examples directory of BluePyEfe 2.

Author of this script: Tanguy Damart @ Blue Brain Project

## Creating the config dictionary

As for BluePyEfe, the first step in extracting e-features is to create a config dictionary. 

The first entry of this dictionary containes the files containing recordings as well as the matching metadata:

In [1]:
import glob

# Get the path of the files for the first cell and build the metadata dictionaries
trace_metadata1 = []
for file in glob.glob("./exp_data/B6/B6_Ch0_IDRest_*.ibw"):
    trace_metadata1.append({
        "i_file": file,
        "v_file": file.replace("Ch0", "Ch3"),
        "i_unit": "pA",
        "v_unit": "mV",
        "t_unit": "s",
        "dt": 0.00025,
        "ljp": 14.
    })

# Do the same for the second cell
trace_metadata2 = []
for file in glob.glob("./exp_data/B8/B8_Ch0_IDRest_*.ibw"):
    trace_metadata2.append({
        "i_file": file,
        "v_file": file.replace("Ch0", "Ch3"),
        "i_unit": "pA",
        "v_unit": "mV",
        "t_unit": "s",
        "dt": 0.00025,
        "ljp": 14.
    })

cells = {
    'MouseNeuron1': {'IDRest': trace_metadata1},
    'MouseNeuron2': {'IDRest': trace_metadata2}
}

The second entry contains the e-features that should be extracted for each protocols as well as the targets (expressed in % of the rheobase):

In [2]:
interesting_efeatures = [
    'Spikecount', 
    'mean_frequency',
    'ISI_CV',
    'AP1_amp',
    'AP_width'
]

protocols = {
    'IDRest': {
        'tolerances': [20.],
        'targets': [150, 200, 250, 300],
        'efeatures': interesting_efeatures,
        'location': 'soma'
    }
}

Finally, we can specify the protocols used to find the rheobase and create the config dictionary:

In [3]:
import pprint
from bluepyemodel.feature_extraction.extract import get_config, extract_efeatures

protocols_threshold = ['IDRest']
threshold_nvalue_save = 1

config_dict = get_config(
    cells,
    protocols,
    protocols_threshold=protocols_threshold,
    threshold_nvalue_save=threshold_nvalue_save,
)

pprint.pprint(config_dict)

{'cells': {'MouseNeuron1': {'IDRest': [{'dt': 0.00025,
                                        'i_file': './exp_data/B6/B6_Ch0_IDRest_182.ibw',
                                        'i_unit': 'pA',
                                        'ljp': 14.0,
                                        't_unit': 's',
                                        'v_file': './exp_data/B6/B6_Ch3_IDRest_182.ibw',
                                        'v_unit': 'mV'},
                                       {'dt': 0.00025,
                                        'i_file': './exp_data/B6/B6_Ch0_IDRest_183.ibw',
                                        'i_unit': 'pA',
                                        'ljp': 14.0,
                                        't_unit': 's',
                                        'v_file': './exp_data/B6/B6_Ch3_IDRest_183.ibw',
                                        'v_unit': 'mV'},
                                       {'dt': 0.00025,
                                     

## Extracting the e-features

In [4]:
efeatures, stimuli, currents = extract_efeatures(
    config_dict, 
    "ExampleCell",
    plot=True
)

<Figure size 432x288 with 0 Axes>

The extract_features function returns three information: the e-features, the stimuli (also called protocols) and the threshold and holding currents:

In [5]:
pprint.pprint(currents)

{'all_hypamp': {'MouseNeuron1': -0.012499390169978142,
                'MouseNeuron2': -0.05312240868806839},
 'all_thresh': {'MouseNeuron1': 0.11874420754611492,
                'MouseNeuron2': 0.09374542534351349},
 'hypamp': [-0.032810899429023266, 0.020311509259045124],
 'thresh': [0.1062448164448142, 0.012499391101300716]}


As the arguement plot is set to True, the experimental traces as well as the efeatures will be plotted and the matching pdfs will be saved under the directory "ExampleCell".